图片预处理及增强

In [1]:
from keras.applications import xception
from keras.preprocessing.image import ImageDataGenerator

def img_preprocessing(val_path,tra_path):
    img_height, img_width=(299,299)
    xception_train_datagen = ImageDataGenerator(zoom_range=[0.8,1.2],
                                           rotation_range=10,
                                           width_shift_range=0.2,
                                           height_shift_range=0.2,
                                           preprocessing_function=xception.preprocess_input)
    xception_validation_datagen = ImageDataGenerator(preprocessing_function=xception.preprocess_input)

    print("")
    print("xception train/validation dataset loading")
    print("------------------------------------")
    print("")
    xception_train_generator = xception_train_datagen.flow_from_directory(
            tra_path,
            target_size=( img_height, img_width),
            batch_size=32,
            class_mode='categorical')
    xception_validation_generator = xception_validation_datagen.flow_from_directory(
            val_path,
            target_size=( img_height, img_width),
            batch_size=32,
            class_mode='categorical',
            shuffle=False)
    return xception_train_generator,xception_validation_generator

Using TensorFlow backend.


模型搭建

In [2]:
from keras.applications.xception import Xception
from keras.preprocessing import image
from keras.models import Model
from keras.layers import GlobalAveragePooling2D,Dense,Dropout
from keras import backend as K
from keras import optimizers
from keras import regularizers
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
def xception_train(path,xception_train_generator,xception_validation_generator):
    xception_base_model = Xception(weights='imagenet', include_top=False)
    x_xception = xception_base_model.output
    x_xception = GlobalAveragePooling2D()(x_xception)
    #x_xception = Dropout(0.4)(x_xception)
    predictions = Dense(10, activation='softmax')(x_xception)
    xception_model = Model(inputs=xception_base_model.input, outputs=predictions)
    for layer in xception_base_model.layers[:249]:
        layer.trainable = False
    for layer in xception_base_model.layers[249:]:
        layer.trainable = True    
    sgd = optimizers.SGD(lr=1e-4, decay=1e-5, momentum=0.9, nesterov=True)
    xception_model.compile(optimizer=sgd, loss='categorical_crossentropy',metrics=['categorical_accuracy'])
    CHECKPOINT_MODEL_SAVE_PATH = os.path.join(path,"xception_best_model_{epoch:04d}_{val_loss:.4f}.hdf5")
    model_check_point =ModelCheckpoint(CHECKPOINT_MODEL_SAVE_PATH,monitor='val_loss',verbose=1,save_best_only=True,save_weights_only=False,period=1) 
    model_early_stop=EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='min')
    print("")
    print("xception training")
    print("------------------------------------")
    print("")
    xception_model.fit_generator(
                            xception_train_generator,
                            epochs=30,
                            validation_data=xception_validation_generator,
                            callbacks=[model_check_point,model_early_stop]
                            )

模型训练

In [ ]:
train_folder = 'train'
test_folder = 'test'
my_train_folder='my_train'
my_validation_folder = 'my_validation'
path_num=5

import os
import glob

#for i in range(path_num):
path_i = os.path.join('t' + str(4))
v_path=os.path.join(path_i,my_validation_folder)
t_path=os.path.join(path_i,my_train_folder)
print(path_i,v_path,t_path)
print("------------------------------------")
xception_train_generator,xception_validation_generator=img_preprocessing(v_path,t_path)
xception_train (path_i,xception_train_generator,xception_validation_generator)

t4 t4\my_validation t4\my_train
------------------------------------

xception train/validation dataset loading
------------------------------------

Found 20408 images belonging to 10 classes.
Found 2016 images belonging to 10 classes.

xception training
------------------------------------

Epoch 1/30
637/638 [============================>.] - ETA: 0s - loss: 2.2941 - categorical_accuracy: 0.1321
Epoch 00001: val_loss improved from inf to 2.29984, saving model to t4\xception_best_model_0001_2.2998.hdf5
638/638 [==============================] - 507s 794ms/step - loss: 2.2941 - categorical_accuracy: 0.1323 - val_loss: 2.2998 - val_categorical_accuracy: 0.1300
Epoch 2/30
637/638 [============================>.] - ETA: 0s - loss: 2.2577 - categorical_accuracy: 0.1755
Epoch 00002: val_loss improved from 2.29984 to 2.28273, saving model to t4\xception_best_model_0002_2.2827.hdf5
638/638 [==============================] - 503s 788ms/step - loss: 2.2577 - categorical_accuracy: 0.1757 - val_